In [1]:
import os
import sys
import grid2op

res = None
try:
    from jyquickhelper import add_notebook_menu
    res = add_notebook_menu()
except ModuleNotFoundError:
    print("Impossible to automatically add a menu / table of content to this notebook.\nYou can download \"jyquickhelper\" package with: \n\"pip install jyquickhelper\"")
res

In [2]:
#创建环境
env = grid2op.make("rte_case14_redisp", test=True)
print(f"grid2op dataset will be downloaded in {os.path.expanduser('~/data_grid2op')}")

D:\virtualenvironment\L2PRN_grid2op\lib\site-packages\grid2op\MakeEnv\Make.py:265: UserWarning: You are using a development environment. This environment is not intended for training agents. It might not be up to date and its primary use if for tests (hence the "test=True" you passed as argument). Use at your own risk.
  warnings.warn(_MAKE_DEV_ENV_WARN)


grid2op dataset will be downloaded in C:\Users\hxj/data_grid2op


In [7]:
#查看本地可用的environment
grid2op.list_available_local_env()

[]

In [8]:
grid2op.list_available_remote_env()

Grid2OpException: Grid2OpException "Impossible to retrieve data at "https://raw.githubusercontent.com/BDonnot/grid2op-datasets/master/datasets.json".
If the problem persists, please contact grid2op developers by sending an issue at https://github.com/rte-france/Grid2Op/issues"

In [4]:
#创建智能体
from grid2op.Agent import DoNothingAgent
my_agent = DoNothingAgent(env.action_space)
#获取agent的性能，性能的获取方式就是reward值，这里采用FlatRewaed，即计算agent在规则下成功操作的时间步数
done = False
time_step = int(0)
cum_reward = 0.
obs = env.reset()
reward = env.reward_range[0]
max_iter = 10
while not done:
    act = my_agent.act(obs, reward, done) # chose an action to do, in this case "do nothing"
    obs, reward, done, info = env.step(act) # implement this action on the powergrid
    cum_reward += reward
    time_step += 1
    if time_step >= max_iter:
        break

print("This agent managed to survive {} timesteps".format(time_step))
print("It's final cumulated reward is {}".format(cum_reward))

This agent managed to survive 10 timesteps
It's final cumulated reward is 11096.61962890625


In [5]:
#另外一种创建方式使用runner，这种方法不需要初始化参数。所有参数设置通过Runner内部main函数完成。
from grid2op.Runner import Runner
runner = Runner(**env.get_params_for_runner(), agentClass=DoNothingAgent)
res = runner.run(nb_episode=1, max_iter=max_iter)
print("The results are:")
for chron_name, _, cum_reward, nb_time_step, max_ts in res:
    msg_tmp = "\tFor chronics located at {}\n".format(chron_name)
    msg_tmp += "\t\t - cumulative reward: {:.2f}\n".format(cum_reward)
    msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}".format(nb_time_step, max_ts)
    print(msg_tmp)

The results are:
	For chronics located at D:\virtualenvironment\L2PRN_grid2op\lib\site-packages\grid2op\data\rte_case14_redisp\chronics\0
		 - cumulative reward: 10948.05
		 - number of time steps completed: 10 / 10


In [6]:
#runner这种方法非常适合比较不同agent的性能，例如，这里引入PowerLinSwitch,该agnet用于仿真连接或者断开传输线所带来的结果
from grid2op.Agent import PowerLineSwitch
runner = Runner(**env.get_params_for_runner(), agentClass=PowerLineSwitch)
res = runner.run(nb_episode=1, max_iter=max_iter)
print("The results are:")
for chron_name, _, cum_reward, nb_time_step, max_ts in res:
    msg_tmp = "\tFor chronics located at {}\n".format(chron_name)
    msg_tmp += "\t\t - cumulative reward: {:.2f}\n".format(cum_reward)
    msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}".format(nb_time_step, max_ts)
    print(msg_tmp)

The results are:
	For chronics located at D:\virtualenvironment\L2PRN_grid2op\lib\site-packages\grid2op\data\rte_case14_redisp\chronics\0
		 - cumulative reward: 10950.26
		 - number of time steps completed: 10 / 10


In [7]:
#也可以通过api来存储agent采取aciton后的详细结果，只需添加main funciton call:path_save
runner = Runner(**env.get_params_for_runner(),
                agentClass=PowerLineSwitch
               )
path_save_expe = os.path.abspath("saved_experiment_donothing")
res = runner.run(nb_episode=1, max_iter=max_iter, path_save=path_save_expe)
print("The results are:")
for chron_name, _, cum_reward, nb_time_step, max_ts in res:
    msg_tmp = "\tFor chronics located at {}\n".format(chron_name)
    msg_tmp += "\t\t - cumulative reward: {:.2f}\n".format(cum_reward)
    msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}".format(nb_time_step, max_ts)
    print(msg_tmp)

The results are:
	For chronics located at D:\virtualenvironment\L2PRN_grid2op\lib\site-packages\grid2op\data\rte_case14_redisp\chronics\0
		 - cumulative reward: 10950.26
		 - number of time steps completed: 10 / 10


In [8]:
os.listdir(os.path.join(path_save_expe, "0"))

['actions.npz',
 'agent_exec_times.npz',
 'disc_lines_cascading_failure.npz',
 'env_modifications.npz',
 'episode_meta.json',
 'episode_times.json',
 'observations.npz',
 'opponent_attack.npz',
 'other_rewards.json',
 'rewards.npz',
 '_parameters.json']